# Pull in the transliteration table

In [0]:
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get('https://scriptsource.org/cms/scripts/page.php?item_id=entry_detail&uid=vsytndbyev', headers=headers)

In [192]:
import pandas as pd

results = pd.read_html(response.text)
df = results[1]
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,NaN,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,NaN,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,NaN,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,NaN,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,NaN,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [193]:
df.Glyph = df.USV.apply(lambda val: chr(int(val, 16)))
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,ሀ,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,ሁ,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,ሂ,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,ሃ,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,ሄ,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [0]:
df.to_csv("raw/transliteration-table.csv", index=False)

# Simplify Transliteration Table

In [195]:
df = pd.read_csv("raw/transliteration-table.csv")
df = df.drop(columns=['USV'])
df = df.set_index('Glyph')
df

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
Glyph,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE
...,...,...,...,...,...,...,...,...,...,...
ᎎ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwē,pWE
ᎏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pW
ፘ,rya,NaN,NaN,NaN,NaN,rya,rya,NaN,~ri,NaN


In [196]:
!pip install Unidecode

In [197]:
import unidecode
import numpy as np

def simplify(value):
    try:
        return unidecode.unidecode(value).lower()
    except:
        return np.nan
    
simplified = df.applymap(simplify)
merged = df.join(simplified, rsuffix='_simp')
merged.head(10)

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA,ALA-LC_simp,Alone-Stokes_simp,B&D_simp,Campbell_simp,Chaîne_simp,Cohen_simp,Dawkins_simp,Dept of State_simp,ethiop_simp,SERA_simp
Glyph,,,,,,,,,,,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he,ha,ha,ha,ha,ha,ha,ha,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE,he,he,he,he,he,he,he,he,he,he
ህ,he/h,h/hi,hə/hø,he,hĕ,h/hə,hï,hɪ,he,h,he/h,h/hi,h@/ho,he,he,h/h@,hi,hi,he,h
ሆ,ho,ho,ho,hō,ho,hʷo,ho,ho,ho,ho,ho,ho,ho,ho,ho,hwo,ho,ho,ho,ho
ለ,la,la,lä,la,lă,lɑ,lä,lə,la,le,la,la,la,la,la,la,la,l@,la,le
ሉ,lu,lu,lu,lū,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu


In [198]:
uniqued = df.fillna('').apply(lambda row: list(set([v for v in row.values if '/' not in str(v)])), axis=1)
uniqued

Glyph
ሀ    [ha, he, hă, hā, hä]
ሁ                [hu, hū]
ሂ                [hi, hī]
ሃ                [ha, hā]
ሄ        [hé, he, hE, hē]
             ...         
ᎎ            [, pwē, pWE]
ᎏ             [, pW, pwe]
ፘ            [, ~ri, rya]
ፙ            [, ~mA, mya]
ፚ            [, fya, ~fi]
Length: 368, dtype: object

In [0]:
uniqued.to_json("raw/lookup-table.json")

# Use to transliterate

In [200]:
import json

with open("raw/lookup-table.json") as f:
    lookup = json.load(f)

punctuation = [
    ['።', '.'],
    ['፡', ' '],
    ['፣', ','],
    ['፤', ';'],
    ['፥', ':'],
    ['፧', '?']
]

for punc in punctuation:
    lookup[punc[0]] = [punc[1]]

lookup

{'ሀ': ['ha', 'he', 'hă', 'hā', 'hä'],
 'ሁ': ['hu', 'hū'],
 'ሂ': ['hi', 'hī'],
 'ሃ': ['ha', 'hā'],
 'ሄ': ['hé', 'he', 'hE', 'hē'],
 'ህ': ['he', 'hï', 'hĕ', 'hɪ', 'h'],
 'ሆ': ['hō', 'ho', 'hʷo'],
 'ለ': ['lə', 'lä', 'la', 'lɑ', 'le', 'lă'],
 'ሉ': ['lu', 'lū'],
 'ሊ': ['li', 'lī'],
 'ላ': ['lā', 'la'],
 'ሌ': ['lē', 'lé', 'le', 'lE', 'lʸe'],
 'ል': ['lĕ', 'lï', 'lɪ', 'le', 'l'],
 'ሎ': ['lʷo', 'lō', 'lo'],
 'ሏ': ['', 'lwā', 'lwä', 'lwa', 'lWa'],
 'ሐ': ['ha', 'He', 'ḥă', 'hā', 'hä', 'ḥa'],
 'ሑ': ['hu', 'Hu', 'ḥu', 'ḥū'],
 'ሒ': ['ḥī', 'hi', 'ḥi', 'hī', 'Hi'],
 'ሓ': ['ḥā', 'ha', 'Ha', 'hā', 'ḥa'],
 'ሔ': ['he', 'HE', 'ḥē', 'hē', 'ḥé', 'ḥe'],
 'ሕ': ['H', 'hï', 'ḥĕ', 'hɪ', 'ḥe'],
 'ሖ': ['ho', 'hʷo', 'ḥo', 'ḥō', 'Ho'],
 'ሗ': ['', 'ḥwā', 'HWa'],
 'መ': ['me', 'ma', 'mä', 'mɑ', 'mă', 'mə'],
 'ሙ': ['mū', 'mu'],
 'ሚ': ['mī', 'mi'],
 'ማ': ['mā', 'ma'],
 'ሜ': ['me', 'mé', 'mE', 'mē', 'mʸe'],
 'ም': ['me', 'm', 'mɪ', 'mĕ', 'mï'],
 'ሞ': ['mo', 'mō', 'mʷo'],
 'ሟ': ['', 'mwä', 'mwa', 'mwā', 'mWa'],
 'ሠ': ['śä', '

In [201]:
lookup_simp = {}
for key in lookup.keys():
    values = list(set([unidecode.unidecode(v).lower() for v in lookup[key]]))
    lookup_simp[key] = [v for v in values if '@' not in v]
    
lookup_simp

{'ሀ': ['ha', 'he'],
 'ሁ': ['hu'],
 'ሂ': ['hi'],
 'ሃ': ['ha'],
 'ሄ': ['he'],
 'ህ': ['he', 'hi', 'h'],
 'ሆ': ['hwo', 'ho'],
 'ለ': ['le', 'la'],
 'ሉ': ['lu'],
 'ሊ': ['li'],
 'ላ': ['la'],
 'ሌ': ['le', 'lye'],
 'ል': ['li', 'le', 'l'],
 'ሎ': ['lwo', 'lo'],
 'ሏ': ['', 'lwa'],
 'ሐ': ['ha', 'he'],
 'ሑ': ['hu'],
 'ሒ': ['hi'],
 'ሓ': ['ha'],
 'ሔ': ['he'],
 'ሕ': ['he', 'hi', 'h'],
 'ሖ': ['hwo', 'ho'],
 'ሗ': ['', 'hwa'],
 'መ': ['me', 'ma'],
 'ሙ': ['mu'],
 'ሚ': ['mi'],
 'ማ': ['ma'],
 'ሜ': ['me', 'mye'],
 'ም': ['me', 'mi', 'm'],
 'ሞ': ['mwo', 'mo'],
 'ሟ': ['', 'mwa'],
 'ሠ': ['sa', "'se"],
 'ሡ': ['su', "'su"],
 'ሢ': ['si', "'si"],
 'ሣ': ["'sa", 'sa'],
 'ሤ': ['sye', "'se", 'se'],
 'ሥ': ['si', "'s", 'se'],
 'ሦ': ['so', 'swo', "'so"],
 'ሧ': ['', "'swa", 'swa'],
 'ረ': ['re', 'ra'],
 'ሩ': ['ru'],
 'ሪ': ['ri'],
 'ራ': ['ra'],
 'ሬ': ['rye', 're'],
 'ር': ['r', 'ri', 're'],
 'ሮ': ['rwo', 'ro'],
 'ሯ': ['', 'rwa'],
 'ሰ': ['sa', 'se'],
 'ሱ': ['su'],
 'ሲ': ['si'],
 'ሳ': ['sa'],
 'ሴ': ['sye', 'se'],
 'ስ': ['si', 's',

In [202]:
# create reverse dictionary
lat_eth = dict()

for key in lookup_simp.keys():
    values = lookup_simp[key]
    if '' in values:
        values.remove('')
    for v in values:
        if v in lat_eth.keys():
            new_list = lat_eth[v].copy()
            new_list.append(key)
            lat_eth[v] = new_list
        else:
            lat_eth[v] = [key]

lat_eth

{' ': ['፡'],
 "'a": ['አ', 'ኣ', 'ዐ', 'ዓ'],
 "'e": ['ኤ', 'እ', 'ዐ', 'ዔ', 'ዕ'],
 "'g": ['ⷝ'],
 "'ga": ['ⷛ'],
 "'ge": ['ⷘ', 'ⷜ'],
 "'gi": ['ⷚ'],
 "'go": ['ⷞ'],
 "'gu": ['ⷙ'],
 "'h": ['ኅ'],
 "'ha": ['ኃ'],
 "'he": ['ኀ', 'ኄ'],
 "'hi": ['ኂ'],
 "'ho": ['ኆ'],
 "'hu": ['ኁ'],
 "'i": ['ኢ', 'ዒ', 'ዕ'],
 "'k": ['ⷍ'],
 "'ka": ['ⷋ'],
 "'ke": ['ⷈ', 'ⷌ'],
 "'ki": ['ⷊ'],
 "'ko": ['ⷎ'],
 "'ku": ['ⷉ'],
 "'o": ['ኦ', 'ዖ'],
 "'q": ['ⷅ'],
 "'qa": ['ⷃ'],
 "'qe": ['ⷀ', 'ⷄ'],
 "'qi": ['ⷂ'],
 "'qo": ['ⷆ'],
 "'qu": ['ⷁ'],
 "'s": ['ሥ', 'ፅ'],
 "'sa": ['ሣ', 'ፃ'],
 "'se": ['ሠ', 'ሤ', 'ፀ', 'ፄ'],
 "'si": ['ሢ', 'ፂ'],
 "'so": ['ሦ', 'ፆ'],
 "'su": ['ሡ', 'ፁ'],
 "'swa": ['ሧ'],
 "'u": ['ኡ', 'ዑ'],
 "'wa": ['ኧ'],
 ',': ['፣'],
 '.': ['።'],
 ':': ['፥'],
 ';': ['፤'],
 '?': ['፧'],
 '`a': ['አ', 'ኣ', 'ዐ', 'ዓ'],
 '`e': ['ኤ', 'እ', 'ዔ', 'ዕ'],
 '`i': ['ኢ', 'ዒ'],
 '`o': ['ኦ', 'ዖ'],
 '`u': ['ኡ', 'ዑ'],
 'a': ['አ', 'ኣ', 'ኧ', 'ዐ', 'ዓ'],
 'a-': ['ኧ'],
 'b': ['ብ'],
 'ba': ['በ', 'ባ'],
 'be': ['በ', 'ቤ', 'ብ'],
 'bi': ['ቢ', 'ብ'],
 'bo': ['ቦ'],
 'bu': ['ቡ

In [0]:
# generate possible transliterations

import string
import itertools

def possibilities(word):
    # split word into chars
    chars = list(word)
    
    # generate all combinations 
    # https://stackoverflow.com/questions/27263155/python-find-all-possible-word-combinations-with-a-sequence-of-characters-word
    combinatorics = itertools.product([True, False], repeat=len(chars) - 1)
    latin_segmentation = []
    add = True
    for combination in combinatorics:
        i = 0
        one_such_combination = [chars[i]]
        for slab in combination:
            i += 1
            if not slab: # there is a join
                one_such_combination[-1] += chars[i]
            else:
                one_such_combination += [chars[i]]
        
        for elmt in one_such_combination:
            if elmt not in lat_eth.keys():
                add = False
                break
        if add == True:  
            latin_segmentation.append(one_such_combination)
            
        # reset
        add = True
    
    # conversion
    ethiopic_opts = []
    for segmentation in latin_segmentation:
        ethiopic_opts += convert(segmentation)
            
    return ethiopic_opts

def convert(segmentation):
    final_list = []
    relevant_lists = []
    for elmt in segmentation:
        relevant_lists.append(lat_eth[elmt])
    for i in itertools.product(*relevant_lists):
        final_list.append(''.join(i))
    return final_list

In [0]:
def prune(possibilities):
    final_possibilities = []
    for candidate in possibilities: 
      if candidate in unique_words:
          final_possibilities.append(candidate)
      elif candidate in punctuation:
        final_possibilities.append(candidate)
    if len(final_possibilities) == 0:
      return possibilities
    return final_possibilities

In [204]:
# TEST

t1 = possibilities('fitireti')

print("Possibilities: ", len(t1))
if "ፍጥረት" in t1:
    print("\nTrue")
else:
    print("\nFalse")

Possibilities:  4320

True


In [18]:
# TEST

t2 = possibilities('inidihi')

print("Possibilities: ", len(t2))
if "እንዲህ" in t2:
    print("\nTrue")
else:
    print("\nFalse")

Possibilities:  9792

True


In [20]:
am_dic_file = open("am_dic.txt", "r", encoding="utf-8")
am_dic = set()

for w in am_dic_file.readlines():
    am_dic.add(w.rstrip())

am_dic_file.close()
print(len(am_dic))

103539


In [21]:
m_verified = []
for candidate in t1: 
    if candidate in am_dic:
        m_verified.append(candidate)
        if candidate == "ፍጥረት":
            print(True)

True


In [22]:
len(m_verified)

1

In [206]:
t3 = possibilities("ameseginalehu")
print(len(t3))
t3_unique = set(t3)
t3_unique = list(t3_unique)
print("Possibilities: ", len(t3_unique))
if "አመሰግናለሁ" in t3:
    print("\nTrue")
else:
    print("\nFalse")

3071250
Possibilities:  3071250

True


In [25]:
t3_verified, truth = true_candidates("አመሰግናለሁ", t3)
print(len(t3_verified))
print(truth)

1
True


In [26]:
print(t3_verified)

['አመሰግናለሁ']


In [115]:
!pip install --user -U nltk

Requirement already up-to-date: nltk in /root/.local/lib/python3.6/site-packages (3.5)


Pre-processing Data
- Replace all numbers/digits with "N"
- Compile dictionary of all unique words and save to "corpus_dic.txt"

In [0]:
punctuation = ["፤", ".", "»", "(", ")", "/", "።", "’", "“", "፣", "!", "”", "‘",\
               "፦", "’", "፡", "&", "፥", "፧"]
def tokenize(word): 
  clean_word = ""
  for n in range(len(word)):
    if word[n] in punctuation:
      clean_word += " "
    elif word[n].isdigit():
      clean_word += " "
    else:
      clean_word += word[n]
  
  clean_word = clean_word.strip()
  clean_words = clean_word.split()
  return clean_words

In [0]:
new_am = open("new-am.txt", "r")
corpus = open("corpus.txt", "w") # corpus = cleaned new_am

unique_words = set()

for line in new_am.readlines():
  words = line.split()
  clean_words = []
  for n in range(len(words)):
    word = words[n]
    if word.isdigit():
      continue
    else: 
      word_tokenized = tokenize(word)
      for w in word_tokenized:
        clean_words.append(w)
        unique_words.add(w)
  corpus.write(" ".join(clean_words))
  corpus.write("\n")

corpus.close()
new_am.close()

corpus_dic = open("corpus_dic.txt", "w")
for word in unique_words:
  corpus_dic.write(word)
  corpus_dic.write("\n")
corpus_dic.close()

Compute n-grams

In [0]:
from nltk.util import ngrams

def extract_ngrams(corpus_file):
  corpus = open(corpus_file, "r", encoding="utf-8")

  uni_dict = {}
  bi_dict = {}
  tri_dict = {}
  ngram_totals = {'uni': 0, 'bi': 0, 'tri': 0}
  for line in corpus.readlines():
    words = line.split()

    # Compute ngrams
    unigrams = ngrams(words, 1)
    bigrams = ngrams(words, 2)
    trigrams = ngrams(words, 3)

    # Calculate frequency of each ngram unit
    for uni_tuple in unigrams:
      uni = uni_tuple[0]
      if uni in uni_dict:
        uni_dict[uni] += 1
      else: 
        uni_dict[uni] = 1
      ngram_totals['uni'] += 1
      
    for bi in bigrams:
      if bi in bi_dict:
        bi_dict[bi] += 1
      else:
        bi_dict[bi] = 1
      ngram_totals['bi'] += 1

    for tri in trigrams:
      if tri in tri_dict:
        tri_dict[tri] += 1
      else:
        tri_dict[tri] = 1
      ngram_totals['tri'] += 1

  return uni_dict, bi_dict, tri_dict, ngram_totals

In [210]:
uni_dict, bi_dict, tri_dict, ngram_totals = extract_ngrams("corpus.txt")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: generator 'ngrams' raised StopIteration
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: generator 'ngrams' raised StopIteration


In [211]:
print("unigram dictionary:", len(uni_dict))
print("bigram dictionary:", len(bi_dict))
print("trigram dictionary:", len(tri_dict))

ngram_dict = [uni_dict, bi_dict, tri_dict]

unigram dictionary: 119481
bigram dictionary: 492127
trigram dictionary: 601817


In [0]:
def calculate_ngram_prob(ngram_dict, ngram_totals, vocab_dict):
  trigrams = ngram_dict[2]
  bigrams = ngram_dict[1]
  unigrams = ngram_dict[0]

  v = len(vocab_dict)

  # trigram prob
  tri_prob_dict = {}
  for (tri, frq) in trigrams.items():
    b = tri[0], tri[1]
    prob = frq + 1/ bigrams[b] + v # add 1 smoothing
    tri_prob_dict[tri] = prob
  
  # bigram prob
  bi_prob_dict = {}
  for (bi, frq) in bigrams.items():
    u = bi[0]
    prob = frq / unigrams[u]
    bi_prob_dict[bi] = prob

  # unigram prob
  uni_prob_dict = {}
  for (uni, frq) in unigrams.items():
    prob = frq / ngram_totals['uni']
    uni_prob_dict[uni] = prob

  return uni_prob_dict, bi_prob_dict, tri_prob_dict

In [0]:
uni_prob_dict, bi_prob_dict, tri_prob_dict = calculate_ngram_prob(ngram_dict, ngram_totals, unique_words)

Calculate word probability based on trigram, bigram, and unigram. Takes a sentence and ngram probability dictionaries to determine the probability of the last word in sentence. 

In [0]:
def extract_word_probabilities(sen, ngram_dict_probabilities):
  probability = {}
  token = sen.split()

  # trigram prob
  tri_dict_prob = ngram_dict_probabilities[2]
  if len(token) >= 3: 
    tri = (token[-3], token[-2], token[-1])
    if tri in tri_dict_prob:
      probability[tri] = tri_dict_prob[tri]
  
  # bigram prob
  bi_dict_prob = ngram_dict_probabilities[1]
  if len(token) >= 2
    bi = (token[-2], token[-1])
    if bi in bi_dict_prob:
      probability[bi] = bi_dict_prob[bi]

  # unigram prob
  uni_dict_prob = ngram_dict_probabilities[0]
  if len(token) >= 1
    uni  = token[-1]
    if uni in uni_dict_prob:
      probability = uni_dict_prob[uni]


Transliteration Pipeline

In [0]:
import operator

# Takes in a text of sentence and returns the highest likelihood
# transliterated sequence.
def transliterate(sentence):
  tokens = sentence.split()
  transliterated_tokens = {}
  for token in tokens:
    all_candidates = possibilities(token)
    valid_candidates = prune(all_candidates)
    if token not in transliterated_tokens:
      transliterated_tokens[token] = valid_candidates

  transliterated_sentence = calculate_best_sequence(tokens, transliterated_tokens)

  return transliterated_sentence

# Takes a list of words in a sentence and a list of all the words' possible
# candidates and determines the best sequence of transliterated words.
def calculate_best_sequence(tokens, transliterated_tokens):
  # Create all possible sentences.
  sentence_candidates = create_all_sentences(tokens, transliterated_tokens)

  # Score all possible sentences. 
  all_scores = {}
  for sentence in sentence_candidates:
    score = calculate_sentence_score(sentence)
    all_scores[sentence] = score

  return max(all_scores.items(), key=operator.itemgetter(1))[0]

# Creates all possible sequences of words and returns a list. 
def create_all_sentences(tokens, transliterated_tokens):
  possible_sentences = {}
  for n in range(len(tokens)): 
    if n == 0:
      possible_sentences[tokens[n]] = [t for t in transliterated_tokens[tokens[n]]]
    else: 
      for t in possible_sentences[tokens[n-1]]:
        for next_t in transliterated_tokens[tokens[n]]:
          new_t = t + " " + next_t
          if tokens[n] in possible_sentences:
            possible_sentences[tokens[n]].append(new_t)
          else:
            possible_sentences[tokens[n]] = [new_t]

  return possible_sentences[tokens[n]]


def calculate_ngram_score(ngram, ngram_dict_prob):
  ngram_list = list(ngram)
  avg_ngram_score = 0
  for ngram in ngram_list:
    if ngram in ngram_dict_prob:
      avg_ngram_score = ngram_dict_prob[ngram]
  return avg_ngram_score/len(ngram_list)

# Takes a sentence (sequential list of words) and returns a likelihood score
# based on its ngram sequences.
def calculate_sentence_score(sentence):
  unigrams = ngrams(sentence, 1)
  bigrams = ngrams(sentence, 2)
  trigrams = ngrams(sentence, 3)

  score = 0
  tri_weight = 2.0
  bi_weight = 1.0
  uni_weight = 0.5

  trigram_score = tri_weight * calculate_ngram_score(trigrams, tri_prob_dict)
  bigram_score = bi_weight * calculate_ngram_score(bigrams, bi_prob_dict)
  unigram_score = uni_weight * calculate_ngram_score(unigrams, uni_prob_dict)

  score += trigram_score + bigram_score + unigram_score
  score = score / len(sentence)

  return score

In [273]:
transliterate("inidihi sil . hulat sewochi liseliyu wade")
#እንዲህ ሲል ። ሁለት ሰዎች ሊጸልዩ ወደ

'እንዲህ ሲል ። ሁለት ሰዎች ሊሤሊዩ ወደ'

In [274]:
transliterate("ijasusme . 'eweneti iwnat `elacehualehu")
#ኢየሱስም ። እውነት እውነት እላችኋለሁ

'ኢየሱስም ። እውነት እውነት እላችኋለሁ'

In [275]:
transliterate("abatune wejeme ina'tun ajakebirm tlalacihu ;")
#አባቱን ወይም እናቱን አያከብርም ትላላችሁ ፤ 

'አባቱን ወይም እናቱን አያከብርም ትላላችሁ ፤'

In [276]:
transliterate("bamine 'ajenetim mote ymoti zendi `enidalawe")
#በምን ዓይነትም ሞት ይሞት ዘንድ እንዳለው

'በምን አዤንትእም ሞተ ይሞት ዘንድ እንዳለው'